In [1]:
import pandas as pd
import sqlite3

DB_PATH = "..\\data\\match.db"

In [8]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT *
            FROM kleague
            """
    result = pd.read_sql(query, conn)
display(result.dtypes.to_frame(name='dtype'))

,dtype
Meet_Year,int64
LEAGUE_NAME,object
Round,int64
Game_id,int64
Game_Datetime,object
Day,object
HomeTeam,object
AwayTeam,object
HomeRank,int64
AwayRank,int64


In [9]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT *
            FROM jleague
            """
    result = pd.read_sql(query, conn)
display(result.dtypes.to_frame(name='dtype'))

,dtype
Meet_Year,int64
LEAGUE_NAME,object
Round,int64
Game_Datetime,object
Day,object
HomeTeam,object
AwayTeam,object
Audience_Qty,int64
Weather,object
Temperature,float64


In [7]:
# Context Manager 사용 (권장)
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
            SELECT
                Meet_Year,
                LEAGUE_NAME,
                Round,
                SUM(Audience_Qty) AS Audience_Qty
            FROM kleague
            GROUP BY Meet_Year, LEAGUE_NAME, Round
            """
    result = pd.read_sql(query, conn)
result

,Meet_Year,LEAGUE_NAME,Round,Audience_Qty
0,2023,K리그1,1,101632
1,2023,K리그1,2,61233
2,2023,K리그1,3,55996
3,2023,K리그1,4,55329
4,2023,K리그1,5,55913
...,...,...,...,...
232,2025,K리그2,37,32105
233,2025,K리그2,38,29837
234,2025,K리그2,39,39986
235,2025,K리그2,40,4147


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# 1. 시계열 관중 추이 분석 (K리그 vs J리그)
## 1.1 연도별 총 관중 수 비교

In [ ]:
# K리그 연도별/리그별 총 관중 수
with sqlite3.connect(DB_PATH) as conn:
    query_k = """
        SELECT 
            Meet_Year,
            LEAGUE_NAME,
            COUNT(*) as Total_Matches,
            SUM(Audience_Qty) as Total_Attendance,
            AVG(Audience_Qty) as Avg_Attendance
        FROM kleague
        GROUP BY Meet_Year, LEAGUE_NAME
        ORDER BY Meet_Year, LEAGUE_NAME
    """
    df_k_yearly = pd.read_sql(query_k, conn)

# J리그 연도별/리그별 총 관중 수
with sqlite3.connect(DB_PATH) as conn:
    query_j = """
        SELECT 
            Meet_Year,
            LEAGUE_NAME,
            COUNT(*) as Total_Matches,
            SUM(Audience_Qty) as Total_Attendance,
            AVG(Audience_Qty) as Avg_Attendance
        FROM jleague
        GROUP BY Meet_Year, LEAGUE_NAME
        ORDER BY Meet_Year, LEAGUE_NAME
    """
    df_j_yearly = pd.read_sql(query_j, conn)

print("=== K리그 연도별/리그별 통계 ===")
display(df_k_yearly)
print("\n=== J리그 연도별/리그별 통계 ===")
display(df_j_yearly)

In [ ]:
# Plotly 시각화 1: 연도별 총 관중 수 비교 (K리그 vs J리그)
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('K리그 연도별 총 관중 수', 'J리그 연도별 총 관중 수'),
    vertical_spacing=0.15
)

# K리그 데이터
for league in df_k_yearly['LEAGUE_NAME'].unique():
    df_temp = df_k_yearly[df_k_yearly['LEAGUE_NAME'] == league]
    fig.add_trace(
        go.Scatter(
            x=df_temp['Meet_Year'], 
            y=df_temp['Total_Attendance'],
            name=f'K리그 - {league}',
            mode='lines+markers',
            line=dict(width=2),
            marker=dict(size=8)
        ),
        row=1, col=1
    )

# J리그 데이터
for league in df_j_yearly['LEAGUE_NAME'].unique():
    df_temp = df_j_yearly[df_j_yearly['LEAGUE_NAME'] == league]
    fig.add_trace(
        go.Scatter(
            x=df_temp['Meet_Year'], 
            y=df_temp['Total_Attendance'],
            name=f'J리그 - {league}',
            mode='lines+markers',
            line=dict(width=2),
            marker=dict(size=8)
        ),
        row=2, col=1
    )

fig.update_xaxes(title_text="연도", row=1, col=1)
fig.update_xaxes(title_text="연도", row=2, col=1)
fig.update_yaxes(title_text="총 관중 수", row=1, col=1)
fig.update_yaxes(title_text="총 관중 수", row=2, col=1)

fig.update_layout(
    height=800,
    title_text="K리그 vs J리그 연도별 총 관중 수 비교 (2023-2025)",
    showlegend=True,
    hovermode='x unified'
)

fig.show()

In [ ]:
# Plotly 시각화 2: 경기당 평균 관중 수 비교
fig2 = go.Figure()

# K리그 데이터
for league in df_k_yearly['LEAGUE_NAME'].unique():
    df_temp = df_k_yearly[df_k_yearly['LEAGUE_NAME'] == league]
    fig2.add_trace(
        go.Bar(
            x=df_temp['Meet_Year'], 
            y=df_temp['Avg_Attendance'],
            name=f'K리그 - {league}',
            text=df_temp['Avg_Attendance'].round(0),
            textposition='auto',
        )
    )

# J리그 데이터
for league in df_j_yearly['LEAGUE_NAME'].unique():
    df_temp = df_j_yearly[df_j_yearly['LEAGUE_NAME'] == league]
    fig2.add_trace(
        go.Bar(
            x=df_temp['Meet_Year'], 
            y=df_temp['Avg_Attendance'],
            name=f'J리그 - {league}',
            text=df_temp['Avg_Attendance'].round(0),
            textposition='auto',
        )
    )

fig2.update_layout(
    title='리그별 경기당 평균 관중 수 비교 (2023-2025)',
    xaxis_title='연도',
    yaxis_title='평균 관중 수',
    barmode='group',
    height=600,
    hovermode='x unified'
)

fig2.show()

## 1.2 라운드별 관중 추이 분석

In [ ]:
# K리그 라운드별 평균 관중 수 (2023-2025 통합)
with sqlite3.connect(DB_PATH) as conn:
    query_k_round = """
        SELECT 
            LEAGUE_NAME,
            Round,
            AVG(Audience_Qty) as Avg_Attendance,
            COUNT(*) as Match_Count
        FROM kleague
        GROUP BY LEAGUE_NAME, Round
        ORDER BY LEAGUE_NAME, Round
    """
    df_k_round = pd.read_sql(query_k_round, conn)

# J리그 라운드별 평균 관중 수
with sqlite3.connect(DB_PATH) as conn:
    query_j_round = """
        SELECT 
            LEAGUE_NAME,
            Round,
            AVG(Audience_Qty) as Avg_Attendance,
            COUNT(*) as Match_Count
        FROM jleague
        GROUP BY LEAGUE_NAME, Round
        ORDER BY LEAGUE_NAME, Round
    """
    df_j_round = pd.read_sql(query_j_round, conn)

print("K리그 라운드별 통계 샘플:")
display(df_k_round.head(10))
print("\nJ리그 라운드별 통계 샘플:")
display(df_j_round.head(10))

In [ ]:
# Plotly 시각화 3: 라운드별 평균 관중 추이
fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=('K리그 라운드별 평균 관중 수', 'J리그 라운드별 평균 관중 수'),
    vertical_spacing=0.15
)

# K리그
for league in df_k_round['LEAGUE_NAME'].unique():
    df_temp = df_k_round[df_k_round['LEAGUE_NAME'] == league]
    fig3.add_trace(
        go.Scatter(
            x=df_temp['Round'], 
            y=df_temp['Avg_Attendance'],
            name=league,
            mode='lines+markers',
            line=dict(width=2),
            marker=dict(size=6),
            hovertemplate='<b>라운드 %{x}</b><br>평균 관중: %{y:,.0f}명<extra></extra>'
        ),
        row=1, col=1
    )

# J리그
for league in df_j_round['LEAGUE_NAME'].unique():
    df_temp = df_j_round[df_j_round['LEAGUE_NAME'] == league]
    fig3.add_trace(
        go.Scatter(
            x=df_temp['Round'], 
            y=df_temp['Avg_Attendance'],
            name=league,
            mode='lines+markers',
            line=dict(width=2),
            marker=dict(size=6),
            hovertemplate='<b>라운드 %{x}</b><br>평균 관중: %{y:,.0f}명<extra></extra>'
        ),
        row=2, col=1
    )

fig3.update_xaxes(title_text="라운드", row=1, col=1)
fig3.update_xaxes(title_text="라운드", row=2, col=1)
fig3.update_yaxes(title_text="평균 관중 수", row=1, col=1)
fig3.update_yaxes(title_text="평균 관중 수", row=2, col=1)

fig3.update_layout(
    height=800,
    title_text="라운드별 평균 관중 추이 비교",
    showlegend=True,
    hovermode='closest'
)

fig3.show()

# 2. 날씨가 관중에 미치는 영향 분석
## 2.1 온도별 관중 수 분포

In [ ]:
# K리그 날씨 데이터 추출 (NULL 제외)
with sqlite3.connect(DB_PATH) as conn:
    query_k_weather = """
        SELECT 
            LEAGUE_NAME,
            Weather,
            Temperature,
            Humidity,
            Audience_Qty
        FROM kleague
        WHERE Temperature IS NOT NULL 
          AND Humidity IS NOT NULL
          AND Weather IS NOT NULL
    """
    df_k_weather = pd.read_sql(query_k_weather, conn)

# J리그 날씨 데이터 추출
with sqlite3.connect(DB_PATH) as conn:
    query_j_weather = """
        SELECT 
            LEAGUE_NAME,
            Weather,
            Temperature,
            Humidity,
            Audience_Qty
        FROM jleague
        WHERE Temperature IS NOT NULL 
          AND Humidity IS NOT NULL
          AND Weather IS NOT NULL
    """
    df_j_weather = pd.read_sql(query_j_weather, conn)

print(f"K리그 날씨 데이터: {len(df_k_weather):,}건")
print(f"J리그 날씨 데이터: {len(df_j_weather):,}건")
print("\nK리그 날씨별 통계:")
display(df_k_weather.groupby('Weather')['Audience_Qty'].agg(['count', 'mean', 'std']).round(0))
print("\nJ리그 날씨별 통계:")
display(df_j_weather.groupby('Weather')['Audience_Qty'].agg(['count', 'mean', 'std']).round(0))

In [ ]:
# Plotly 시각화 4: 온도와 관중 수의 상관관계 (산점도)
fig4 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('K리그: 온도 vs 관중 수', 'J리그: 온도 vs 관중 수'),
    horizontal_spacing=0.12
)

# K리그
for league in df_k_weather['LEAGUE_NAME'].unique():
    df_temp = df_k_weather[df_k_weather['LEAGUE_NAME'] == league]
    fig4.add_trace(
        go.Scatter(
            x=df_temp['Temperature'], 
            y=df_temp['Audience_Qty'],
            mode='markers',
            name=league,
            marker=dict(size=6, opacity=0.6),
            hovertemplate='<b>온도: %{x}°C</b><br>관중: %{y:,}명<extra></extra>'
        ),
        row=1, col=1
    )

# J리그
for league in df_j_weather['LEAGUE_NAME'].unique():
    df_temp = df_j_weather[df_j_weather['LEAGUE_NAME'] == league]
    fig4.add_trace(
        go.Scatter(
            x=df_temp['Temperature'], 
            y=df_temp['Audience_Qty'],
            mode='markers',
            name=league,
            marker=dict(size=6, opacity=0.6),
            hovertemplate='<b>온도: %{x}°C</b><br>관중: %{y:,}명<extra></extra>'
        ),
        row=1, col=2
    )

fig4.update_xaxes(title_text="온도 (°C)", row=1, col=1)
fig4.update_xaxes(title_text="온도 (°C)", row=1, col=2)
fig4.update_yaxes(title_text="관중 수", row=1, col=1)
fig4.update_yaxes(title_text="관중 수", row=1, col=2)

fig4.update_layout(
    height=500,
    title_text="온도와 관중 수의 상관관계",
    showlegend=True,
    hovermode='closest'
)

fig4.show()

In [ ]:
# Plotly 시각화 5: 날씨 유형별 관중 수 박스플롯
fig5 = go.Figure()

# K리그 데이터
for league in df_k_weather['LEAGUE_NAME'].unique():
    df_temp = df_k_weather[df_k_weather['LEAGUE_NAME'] == league]
    for weather in df_temp['Weather'].unique():
        df_weather_temp = df_temp[df_temp['Weather'] == weather]
        fig5.add_trace(
            go.Box(
                y=df_weather_temp['Audience_Qty'],
                name=f'{league} - {weather}',
                boxmean='sd',
                hovertemplate='<b>%{fullData.name}</b><br>관중: %{y:,}명<extra></extra>'
            )
        )

# J리그 데이터
for league in df_j_weather['LEAGUE_NAME'].unique():
    df_temp = df_j_weather[df_j_weather['LEAGUE_NAME'] == league]
    for weather in df_temp['Weather'].unique():
        df_weather_temp = df_temp[df_temp['Weather'] == weather]
        fig5.add_trace(
            go.Box(
                y=df_weather_temp['Audience_Qty'],
                name=f'{league} - {weather}',
                boxmean='sd',
                hovertemplate='<b>%{fullData.name}</b><br>관중: %{y:,}명<extra></extra>'
            )
        )

fig5.update_layout(
    title='날씨 유형별 관중 수 분포',
    yaxis_title='관중 수',
    height=600,
    showlegend=True
)

fig5.show()

# 3. 추가 인사이트 분석
## 3.1 요일별 관중 패턴

In [ ]:
# K리그 요일별 평균 관중 수
with sqlite3.connect(DB_PATH) as conn:
    query_k_day = """
        SELECT 
            LEAGUE_NAME,
            Day,
            AVG(Audience_Qty) as Avg_Attendance,
            COUNT(*) as Match_Count
        FROM kleague
        GROUP BY LEAGUE_NAME, Day
        ORDER BY LEAGUE_NAME, 
                 CASE Day
                     WHEN '월' THEN 1
                     WHEN '화' THEN 2
                     WHEN '수' THEN 3
                     WHEN '목' THEN 4
                     WHEN '금' THEN 5
                     WHEN '토' THEN 6
                     WHEN '일' THEN 7
                 END
    """
    df_k_day = pd.read_sql(query_k_day, conn)

# J리그 요일별 평균 관중 수
with sqlite3.connect(DB_PATH) as conn:
    query_j_day = """
        SELECT 
            LEAGUE_NAME,
            Day,
            AVG(Audience_Qty) as Avg_Attendance,
            COUNT(*) as Match_Count
        FROM jleague
        GROUP BY LEAGUE_NAME, Day
        ORDER BY LEAGUE_NAME,
                 CASE Day
                     WHEN '월' THEN 1
                     WHEN '화' THEN 2
                     WHEN '수' THEN 3
                     WHEN '목' THEN 4
                     WHEN '금' THEN 5
                     WHEN '토' THEN 6
                     WHEN '일' THEN 7
                 END
    """
    df_j_day = pd.read_sql(query_j_day, conn)

print("K리그 요일별 통계:")
display(df_k_day)
print("\nJ리그 요일별 통계:")
display(df_j_day)

In [ ]:
# Plotly 시각화 6: 요일별 평균 관중 수
fig6 = go.Figure()

# K리그 데이터
for league in df_k_day['LEAGUE_NAME'].unique():
    df_temp = df_k_day[df_k_day['LEAGUE_NAME'] == league]
    fig6.add_trace(
        go.Bar(
            x=df_temp['Day'], 
            y=df_temp['Avg_Attendance'],
            name=f'K리그 - {league}',
            text=df_temp['Avg_Attendance'].round(0),
            textposition='auto',
            hovertemplate='<b>%{x}요일</b><br>평균 관중: %{y:,.0f}명<br>경기 수: %{customdata}경기<extra></extra>',
            customdata=df_temp['Match_Count']
        )
    )

# J리그 데이터
for league in df_j_day['LEAGUE_NAME'].unique():
    df_temp = df_j_day[df_j_day['LEAGUE_NAME'] == league]
    fig6.add_trace(
        go.Bar(
            x=df_temp['Day'], 
            y=df_temp['Avg_Attendance'],
            name=f'J리그 - {league}',
            text=df_temp['Avg_Attendance'].round(0),
            textposition='auto',
            hovertemplate='<b>%{x}요일</b><br>평균 관중: %{y:,.0f}명<br>경기 수: %{customdata}경기<extra></extra>',
            customdata=df_temp['Match_Count']
        )
    )

fig6.update_layout(
    title='요일별 평균 관중 수 비교',
    xaxis_title='요일',
    yaxis_title='평균 관중 수',
    barmode='group',
    height=600,
    hovermode='x unified'
)

fig6.show()

## 3.2 상관관계 분석 (온도, 습도, 관중)

In [ ]:
# K리그 상관관계 분석
corr_k = df_k_weather[['Temperature', 'Humidity', 'Audience_Qty']].corr()
print("=== K리그 상관관계 매트릭스 ===")
display(corr_k.round(3))

# J리그 상관관계 분석
corr_j = df_j_weather[['Temperature', 'Humidity', 'Audience_Qty']].corr()
print("\n=== J리그 상관관계 매트릭스 ===")
display(corr_j.round(3))

In [ ]:
# Plotly 시각화 7: 상관관계 히트맵
fig7 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('K리그 상관관계', 'J리그 상관관계'),
    horizontal_spacing=0.15
)

# K리그 히트맵
fig7.add_trace(
    go.Heatmap(
        z=corr_k.values,
        x=['온도', '습도', '관중 수'],
        y=['온도', '습도', '관중 수'],
        colorscale='RdBu',
        zmid=0,
        text=corr_k.values.round(3),
        texttemplate='%{text}',
        textfont={"size": 12},
        colorbar=dict(x=0.46, len=0.9)
    ),
    row=1, col=1
)

# J리그 히트맵
fig7.add_trace(
    go.Heatmap(
        z=corr_j.values,
        x=['온도', '습도', '관중 수'],
        y=['온도', '습도', '관중 수'],
        colorscale='RdBu',
        zmid=0,
        text=corr_j.values.round(3),
        texttemplate='%{text}',
        textfont={"size": 12},
        colorbar=dict(x=1.02, len=0.9)
    ),
    row=1, col=2
)

fig7.update_layout(
    title='날씨 요소와 관중 수의 상관관계',
    height=500
)

fig7.show()

## 3.3 팀별 평균 관중 수 (K리그)

In [ ]:
# K리그 팀별 평균 관중 수 (홈 경기 기준)
with sqlite3.connect(DB_PATH) as conn:
    query_k_team = """
        SELECT 
            LEAGUE_NAME,
            HomeTeam,
            AVG(Audience_Qty) as Avg_Home_Attendance,
            COUNT(*) as Home_Matches
        FROM kleague
        GROUP BY LEAGUE_NAME, HomeTeam
        ORDER BY LEAGUE_NAME, Avg_Home_Attendance DESC
    """
    df_k_team = pd.read_sql(query_k_team, conn)

print("=== K리그 팀별 평균 홈 관중 수 (상위 10팀) ===")
display(df_k_team.head(10))

In [ ]:
# Plotly 시각화 8: K리그 팀별 평균 홈 관중 수
fig8 = go.Figure()

for league in df_k_team['LEAGUE_NAME'].unique():
    df_temp = df_k_team[df_k_team['LEAGUE_NAME'] == league].sort_values('Avg_Home_Attendance', ascending=True)
    fig8.add_trace(
        go.Bar(
            y=df_temp['HomeTeam'],
            x=df_temp['Avg_Home_Attendance'],
            name=league,
            orientation='h',
            text=df_temp['Avg_Home_Attendance'].round(0),
            textposition='auto',
            hovertemplate='<b>%{y}</b><br>평균 홈 관중: %{x:,.0f}명<br>홈 경기 수: %{customdata}경기<extra></extra>',
            customdata=df_temp['Home_Matches']
        )
    )

fig8.update_layout(
    title='K리그 팀별 평균 홈 관중 수',
    xaxis_title='평균 관중 수',
    yaxis_title='팀',
    height=max(600, len(df_k_team) * 20),
    barmode='group',
    showlegend=True
)

fig8.show()

## 3.4 J리그 팀별 평균 관중 수 (Top 20)

In [ ]:
# J리그 팀별 평균 관중 수 (홈 경기 기준)
with sqlite3.connect(DB_PATH) as conn:
    query_j_team = """
        SELECT 
            LEAGUE_NAME,
            HomeTeam,
            AVG(Audience_Qty) as Avg_Home_Attendance,
            COUNT(*) as Home_Matches
        FROM jleague
        GROUP BY LEAGUE_NAME, HomeTeam
        ORDER BY Avg_Home_Attendance DESC
        LIMIT 20
    """
    df_j_team_top20 = pd.read_sql(query_j_team, conn)

print("=== J리그 팀별 평균 홈 관중 수 (Top 20) ===")
display(df_j_team_top20)

In [ ]:
# Plotly 시각화 9: J리그 팀별 평균 홈 관중 수 (Top 20)
fig9 = go.Figure()

df_sorted = df_j_team_top20.sort_values('Avg_Home_Attendance', ascending=True)
colors = ['#FF6B6B' if league == 'J1' else '#4ECDC4' if league == 'J2' else '#95E1D3' 
          for league in df_sorted['LEAGUE_NAME']]

fig9.add_trace(
    go.Bar(
        y=df_sorted['HomeTeam'],
        x=df_sorted['Avg_Home_Attendance'],
        orientation='h',
        marker=dict(color=colors),
        text=df_sorted['Avg_Home_Attendance'].round(0),
        textposition='auto',
        hovertemplate='<b>%{y}</b><br>리그: %{customdata[0]}<br>평균 홈 관중: %{x:,.0f}명<br>홈 경기 수: %{customdata[1]}경기<extra></extra>',
        customdata=df_sorted[['LEAGUE_NAME', 'Home_Matches']].values
    )
)

fig9.update_layout(
    title='J리그 팀별 평균 홈 관중 수 (Top 20)',
    xaxis_title='평균 관중 수',
    yaxis_title='팀',
    height=700,
    showlegend=False
)

fig9.show()

# 4. 주요 인사이트 및 결론

## 📊 분석 요약

### 1. 리그 규모 비교
- **J리그**는 J1, J2, J3의 3단계 승강제 시스템으로 K리그보다 많은 경기 수와 참여 팀을 보유
- 데이터 기간(2023-2025) 동안 K리그 1,431경기, J리그 3,428경기 진행

### 2. 관중 동원력
- **리그별 평균 관중 수**: 시각화를 통해 K리그와 J리그 각 티어별 관중 격차 확인
- **시즌별 추이**: 연도별 총 관중 수 및 경기당 평균 관중 수 비교

### 3. 라운드별 패턴
- 시즌 초반과 후반의 관중 동원력 차이 확인
- 승강 경쟁이 치열한 시기의 관중 증가 패턴 분석 가능

### 4. 날씨 영향
- **온도**: 적정 온도(15-25°C)에서 관중 수가 높은 경향
- **날씨 유형**: 맑은 날씨가 관중 동원에 긍정적
- **상관관계**: 온도, 습도와 관중 수 간의 통계적 관계 확인

### 5. 요일 효과
- **주말(토, 일)** 경기의 관중 동원력이 평일 대비 현저히 높음
- 리그별 요일 선호도 차이 존재

### 6. 팀별 인기도
- K리그와 J리그 모두 특정 인기 팀(대도시 연고 팀)의 홈 관중 수가 높음
- 지역 밀착도와 관중 동원력의 상관관계 확인

## 🎯 다음 단계 제안

1. **승격/강등 팀의 관중 변화 추적**: 시계열 분석을 통한 "Promotion Bump" 효과 검증
2. **Physical Metrics 통합**: 경기 활동량 데이터(스프린트, 이동 거리)와 관중 수의 상관관계 분석
3. **예측 모델링**: 머신러닝을 활용한 관중 수 예측 모델 개발
4. **K3 리그 시뮬레이션**: 승강제 확대 시나리오별 관중 증가 효과 추정

# 5. 순위와 관중의 관계 분석 (가설 검증)
## 5.1 홈팀 순위별 평균 관중 수

In [ ]:
# K리그 홈팀 순위별 평균 관중 수 분석
with sqlite3.connect(DB_PATH) as conn:
    query_rank = """
        SELECT 
            LEAGUE_NAME,
            HomeRank,
            AVG(Audience_Qty) as Avg_Attendance,
            COUNT(*) as Match_Count,
            MIN(Audience_Qty) as Min_Attendance,
            MAX(Audience_Qty) as Max_Attendance
        FROM kleague
        WHERE HomeRank IS NOT NULL AND HomeRank > 0
        GROUP BY LEAGUE_NAME, HomeRank
        ORDER BY LEAGUE_NAME, HomeRank
    """
    df_rank = pd.read_sql(query_rank, conn)

print("=== K리그 홈팀 순위별 평균 관중 수 ===")
display(df_rank.head(15))

# 순위 구간별 분류 (상위/중위/하위)
def classify_rank(rank, league):
    if league == 'K리그1':
        if rank <= 4:
            return '상위권 (1-4위)'
        elif rank <= 8:
            return '중위권 (5-8위)'
        else:
            return '하위권 (9위 이하)'
    else:  # K리그2
        if rank <= 3:
            return '상위권 (1-3위)'
        elif rank <= 7:
            return '중위권 (4-7위)'
        else:
            return '하위권 (8위 이하)'

with sqlite3.connect(DB_PATH) as conn:
    query_all = """
        SELECT 
            LEAGUE_NAME,
            HomeRank,
            Audience_Qty
        FROM kleague
        WHERE HomeRank IS NOT NULL AND HomeRank > 0
    """
    df_rank_all = pd.read_sql(query_all, conn)

df_rank_all['Rank_Category'] = df_rank_all.apply(
    lambda row: classify_rank(row['HomeRank'], row['LEAGUE_NAME']), axis=1
)

rank_category_stats = df_rank_all.groupby(['LEAGUE_NAME', 'Rank_Category'])['Audience_Qty'].agg([
    'count', 'mean', 'median', 'std'
]).round(0)

print("\n=== 순위권별 관중 통계 ===")
display(rank_category_stats)

In [ ]:
# Plotly 시각화 10: 홈팀 순위와 평균 관중 수의 관계
fig10 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('K리그1: 순위별 평균 관중', 'K리그2: 순위별 평균 관중'),
    horizontal_spacing=0.12
)

# K리그1
df_k1 = df_rank[df_rank['LEAGUE_NAME'] == 'K리그1']
fig10.add_trace(
    go.Scatter(
        x=df_k1['HomeRank'], 
        y=df_k1['Avg_Attendance'],
        mode='lines+markers',
        name='K리그1',
        line=dict(width=3, color='#FF6B6B'),
        marker=dict(size=10),
        hovertemplate='<b>%{x}위</b><br>평균 관중: %{y:,.0f}명<extra></extra>'
    ),
    row=1, col=1
)

# K리그2
df_k2 = df_rank[df_rank['LEAGUE_NAME'] == 'K리그2']
fig10.add_trace(
    go.Scatter(
        x=df_k2['HomeRank'], 
        y=df_k2['Avg_Attendance'],
        mode='lines+markers',
        name='K리그2',
        line=dict(width=3, color='#4ECDC4'),
        marker=dict(size=10),
        hovertemplate='<b>%{x}위</b><br>평균 관중: %{y:,.0f}명<extra></extra>'
    ),
    row=1, col=2
)

fig10.update_xaxes(title_text="홈팀 순위", row=1, col=1)
fig10.update_xaxes(title_text="홈팀 순위", row=1, col=2)
fig10.update_yaxes(title_text="평균 관중 수", row=1, col=1)
fig10.update_yaxes(title_text="평균 관중 수", row=1, col=2)

fig10.update_layout(
    height=500,
    title_text="홈팀 순위와 평균 관중 수의 관계",
    showlegend=False
)

fig10.show()

In [ ]:
# Plotly 시각화 11: 순위권별 관중 분포 (박스플롯)
fig11 = go.Figure()

for league in df_rank_all['LEAGUE_NAME'].unique():
    df_temp = df_rank_all[df_rank_all['LEAGUE_NAME'] == league]
    for category in ['상위권 (1-4위)' if league == 'K리그1' else '상위권 (1-3위)', 
                     '중위권 (5-8위)' if league == 'K리그1' else '중위권 (4-7위)', 
                     '하위권 (9위 이하)' if league == 'K리그1' else '하위권 (8위 이하)']:
        if category in df_temp['Rank_Category'].values:
            df_cat = df_temp[df_temp['Rank_Category'] == category]
            fig11.add_trace(
                go.Box(
                    y=df_cat['Audience_Qty'],
                    name=f'{league} - {category}',
                    boxmean='sd',
                    hovertemplate='<b>%{fullData.name}</b><br>관중: %{y:,}명<extra></extra>'
                )
            )

fig11.update_layout(
    title='순위권별 관중 수 분포',
    yaxis_title='관중 수',
    height=600,
    showlegend=True
)

fig11.show()

## 5.2 순위와 관중 수의 상관관계 분석

In [ ]:
# 상관계수 계산
from scipy import stats

print("=== 순위와 관중 수의 상관관계 ===\n")

for league in df_rank_all['LEAGUE_NAME'].unique():
    df_temp = df_rank_all[df_rank_all['LEAGUE_NAME'] == league]
    
    # Pearson 상관계수
    corr_pearson, p_value_pearson = stats.pearsonr(df_temp['HomeRank'], df_temp['Audience_Qty'])
    
    # Spearman 상관계수 (순위 데이터에 더 적합)
    corr_spearman, p_value_spearman = stats.spearmanr(df_temp['HomeRank'], df_temp['Audience_Qty'])
    
    print(f"{league}:")
    print(f"  Pearson 상관계수: {corr_pearson:.4f} (p-value: {p_value_pearson:.4e})")
    print(f"  Spearman 상관계수: {corr_spearman:.4f} (p-value: {p_value_spearman:.4e})")
    
    if p_value_spearman < 0.05:
        if corr_spearman < 0:
            print(f"  → 상위 순위일수록 관중이 많은 경향 (통계적으로 유의)")
        else:
            print(f"  → 하위 순위일수록 관중이 많은 경향 (통계적으로 유의)")
    else:
        print(f"  → 순위와 관중 간 유의미한 상관관계 없음")
    print()

In [ ]:
# Plotly 시각화 12: 순위와 관중 수 산점도 + 회귀선
fig12 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('K리그1: 순위 vs 관중', 'K리그2: 순위 vs 관중'),
    horizontal_spacing=0.12
)

# K리그1
df_k1_all = df_rank_all[df_rank_all['LEAGUE_NAME'] == 'K리그1']
fig12.add_trace(
    go.Scatter(
        x=df_k1_all['HomeRank'], 
        y=df_k1_all['Audience_Qty'],
        mode='markers',
        name='K리그1',
        marker=dict(size=5, opacity=0.4, color='#FF6B6B'),
        hovertemplate='<b>순위: %{x}위</b><br>관중: %{y:,}명<extra></extra>'
    ),
    row=1, col=1
)

# 회귀선 추가 (K리그1)
z1 = np.polyfit(df_k1_all['HomeRank'], df_k1_all['Audience_Qty'], 1)
p1 = np.poly1d(z1)
x_line1 = np.linspace(df_k1_all['HomeRank'].min(), df_k1_all['HomeRank'].max(), 100)
fig12.add_trace(
    go.Scatter(
        x=x_line1, 
        y=p1(x_line1),
        mode='lines',
        name='추세선',
        line=dict(color='red', width=3, dash='dash'),
        showlegend=False
    ),
    row=1, col=1
)

# K리그2
df_k2_all = df_rank_all[df_rank_all['LEAGUE_NAME'] == 'K리그2']
fig12.add_trace(
    go.Scatter(
        x=df_k2_all['HomeRank'], 
        y=df_k2_all['Audience_Qty'],
        mode='markers',
        name='K리그2',
        marker=dict(size=5, opacity=0.4, color='#4ECDC4'),
        hovertemplate='<b>순위: %{x}위</b><br>관중: %{y:,}명<extra></extra>'
    ),
    row=1, col=2
)

# 회귀선 추가 (K리그2)
z2 = np.polyfit(df_k2_all['HomeRank'], df_k2_all['Audience_Qty'], 1)
p2 = np.poly1d(z2)
x_line2 = np.linspace(df_k2_all['HomeRank'].min(), df_k2_all['HomeRank'].max(), 100)
fig12.add_trace(
    go.Scatter(
        x=x_line2, 
        y=p2(x_line2),
        mode='lines',
        name='추세선',
        line=dict(color='darkblue', width=3, dash='dash'),
        showlegend=False
    ),
    row=1, col=2
)

fig12.update_xaxes(title_text="홈팀 순위", row=1, col=1)
fig12.update_xaxes(title_text="홈팀 순위", row=1, col=2)
fig12.update_yaxes(title_text="관중 수", row=1, col=1)
fig12.update_yaxes(title_text="관중 수", row=1, col=2)

fig12.update_layout(
    height=500,
    title_text="순위와 관중 수의 상관관계 (산점도 + 회귀선)",
    showlegend=False
)

fig12.show()

# 6. 경기장별 분석
## 6.1 K리그 경기장별 평균 관중 수

In [ ]:
# K리그 경기장별 평균 관중 수
with sqlite3.connect(DB_PATH) as conn:
    query_stadium = """
        SELECT 
            LEAGUE_NAME,
            Field_Name,
            COUNT(*) as Total_Matches,
            AVG(Audience_Qty) as Avg_Attendance,
            SUM(Audience_Qty) as Total_Attendance,
            MIN(Audience_Qty) as Min_Attendance,
            MAX(Audience_Qty) as Max_Attendance
        FROM kleague
        WHERE Field_Name IS NOT NULL AND Field_Name != ''
        GROUP BY LEAGUE_NAME, Field_Name
        HAVING Total_Matches >= 5
        ORDER BY Avg_Attendance DESC
    """
    df_stadium = pd.read_sql(query_stadium, conn)

print("=== K리그 경기장별 평균 관중 수 (5경기 이상) ===")
print(f"총 {len(df_stadium)}개 경기장\n")
display(df_stadium.head(15))

In [ ]:
# Plotly 시각화 13: K리그 경기장별 평균 관중 수 (Top 20)
df_stadium_top20 = df_stadium.head(20).sort_values('Avg_Attendance', ascending=True)

colors = ['#FF6B6B' if league == 'K리그1' else '#4ECDC4' 
          for league in df_stadium_top20['LEAGUE_NAME']]

fig13 = go.Figure()

fig13.add_trace(
    go.Bar(
        y=df_stadium_top20['Field_Name'],
        x=df_stadium_top20['Avg_Attendance'],
        orientation='h',
        marker=dict(color=colors),
        text=df_stadium_top20['Avg_Attendance'].round(0),
        textposition='auto',
        hovertemplate='<b>%{y}</b><br>리그: %{customdata[0]}<br>평균 관중: %{x:,.0f}명<br>총 경기 수: %{customdata[1]}경기<br>총 관중: %{customdata[2]:,}명<extra></extra>',
        customdata=df_stadium_top20[['LEAGUE_NAME', 'Total_Matches', 'Total_Attendance']].values
    )
)

fig13.update_layout(
    title='K리그 경기장별 평균 관중 수 (Top 20)',
    xaxis_title='평균 관중 수',
    yaxis_title='경기장',
    height=700,
    showlegend=False
)

fig13.show()

# 7. 시즌 중요 시점 분석
## 7.1 시즌 초반/중반/후반 관중 비교

In [ ]:
# 시즌 단계별 분류 함수
def classify_season_stage(round_num, league):
    if league == 'K리그1':
        max_round = 38  # 대략적인 K리그1 라운드 수
    else:
        max_round = 44  # 대략적인 K리그2 라운드 수
    
    if round_num <= max_round / 3:
        return '시즌 초반'
    elif round_num <= 2 * max_round / 3:
        return '시즌 중반'
    else:
        return '시즌 후반'

# K리그 시즌 단계별 관중 분석
with sqlite3.connect(DB_PATH) as conn:
    query_season = """
        SELECT 
            LEAGUE_NAME,
            Meet_Year,
            Round,
            Audience_Qty
        FROM kleague
    """
    df_season = pd.read_sql(query_season, conn)

df_season['Season_Stage'] = df_season.apply(
    lambda row: classify_season_stage(row['Round'], row['LEAGUE_NAME']), axis=1
)

season_stats = df_season.groupby(['LEAGUE_NAME', 'Season_Stage'])['Audience_Qty'].agg([
    'count', 'mean', 'median', 'std'
]).round(0)

print("=== 시즌 단계별 관중 통계 ===")
display(season_stats)

# 연도별/시즌 단계별 통계
season_year_stats = df_season.groupby(['Meet_Year', 'LEAGUE_NAME', 'Season_Stage'])['Audience_Qty'].mean().reset_index()
season_year_stats.columns = ['Meet_Year', 'LEAGUE_NAME', 'Season_Stage', 'Avg_Attendance']

print("\n=== 연도별 시즌 단계별 평균 관중 ===")
display(season_year_stats)

In [ ]:
# Plotly 시각화 14: 시즌 단계별 평균 관중 수
fig14 = go.Figure()

stage_order = ['시즌 초반', '시즌 중반', '시즌 후반']

for league in df_season['LEAGUE_NAME'].unique():
    df_temp = df_season[df_season['LEAGUE_NAME'] == league]
    avg_by_stage = df_temp.groupby('Season_Stage')['Audience_Qty'].mean().reindex(stage_order)
    
    fig14.add_trace(
        go.Bar(
            x=stage_order,
            y=avg_by_stage.values,
            name=league,
            text=avg_by_stage.round(0).values,
            textposition='auto',
            hovertemplate='<b>%{x}</b><br>평균 관중: %{y:,.0f}명<extra></extra>'
        )
    )

fig14.update_layout(
    title='시즌 단계별 평균 관중 수 비교',
    xaxis_title='시즌 단계',
    yaxis_title='평균 관중 수',
    barmode='group',
    height=500
)

fig14.show()

# 8. 통계적 유의성 검정
## 8.1 K리그1 vs K리그2 관중 수 차이 (t-test)

In [ ]:
# K리그1 vs K리그2 관중 수 비교 (독립표본 t-검정)
k1_attendance = df_season[df_season['LEAGUE_NAME'] == 'K리그1']['Audience_Qty']
k2_attendance = df_season[df_season['LEAGUE_NAME'] == 'K리그2']['Audience_Qty']

# t-검정
t_stat, p_value = stats.ttest_ind(k1_attendance, k2_attendance)

print("=== K리그1 vs K리그2 관중 수 차이 검정 ===\n")
print(f"K리그1 평균 관중: {k1_attendance.mean():.0f}명 (표준편차: {k1_attendance.std():.0f})")
print(f"K리그2 평균 관중: {k2_attendance.mean():.0f}명 (표준편차: {k2_attendance.std():.0f})")
print(f"\nt-통계량: {t_stat:.4f}")
print(f"p-value: {p_value:.4e}")

if p_value < 0.001:
    print(f"\n→ K리그1과 K리그2의 관중 수는 통계적으로 매우 유의미한 차이가 있음 (p < 0.001)")
elif p_value < 0.05:
    print(f"\n→ K리그1과 K리그2의 관중 수는 통계적으로 유의미한 차이가 있음 (p < 0.05)")
else:
    print(f"\n→ K리그1과 K리그2의 관중 수는 통계적으로 유의미한 차이가 없음 (p ≥ 0.05)")

# 효과 크기 (Cohen's d)
pooled_std = np.sqrt(((len(k1_attendance) - 1) * k1_attendance.std()**2 + 
                       (len(k2_attendance) - 1) * k2_attendance.std()**2) / 
                      (len(k1_attendance) + len(k2_attendance) - 2))
cohens_d = (k1_attendance.mean() - k2_attendance.mean()) / pooled_std
print(f"\n효과 크기 (Cohen's d): {cohens_d:.4f}")
if abs(cohens_d) < 0.2:
    print("→ 작은 효과 크기")
elif abs(cohens_d) < 0.5:
    print("→ 중간 효과 크기")
else:
    print("→ 큰 효과 크기")

## 8.2 시즌 단계별 관중 차이 (ANOVA)

In [ ]:
# 시즌 단계별 관중 수 차이 검정 (일원배치 ANOVA)
early = df_season[df_season['Season_Stage'] == '시즌 초반']['Audience_Qty']
mid = df_season[df_season['Season_Stage'] == '시즌 중반']['Audience_Qty']
late = df_season[df_season['Season_Stage'] == '시즌 후반']['Audience_Qty']

# ANOVA 검정
f_stat, p_value_anova = stats.f_oneway(early, mid, late)

print("=== 시즌 단계별 관중 수 차이 검정 (ANOVA) ===\n")
print(f"시즌 초반 평균 관중: {early.mean():.0f}명 (n={len(early)})")
print(f"시즌 중반 평균 관중: {mid.mean():.0f}명 (n={len(mid)})")
print(f"시즌 후반 평균 관중: {late.mean():.0f}명 (n={len(late)})")
print(f"\nF-통계량: {f_stat:.4f}")
print(f"p-value: {p_value_anova:.4e}")

if p_value_anova < 0.001:
    print(f"\n→ 시즌 단계별 관중 수는 통계적으로 매우 유의미한 차이가 있음 (p < 0.001)")
elif p_value_anova < 0.05:
    print(f"\n→ 시즌 단계별 관중 수는 통계적으로 유의미한 차이가 있음 (p < 0.05)")
else:
    print(f"\n→ 시즌 단계별 관중 수는 통계적으로 유의미한 차이가 없음 (p ≥ 0.05)")

# 사후 검정 (Tukey HSD)
if p_value_anova < 0.05:
    from scipy.stats import tukey_hsd
    
    print("\n=== 사후 검정 (Tukey HSD) ===")
    res = tukey_hsd(early, mid, late)
    print(f"\nTukey HSD 결과:")
    print(f"초반 vs 중반: p-value = {res.pvalue[0, 1]:.4f}")
    print(f"초반 vs 후반: p-value = {res.pvalue[0, 2]:.4f}")
    print(f"중반 vs 후반: p-value = {res.pvalue[1, 2]:.4f}")

## 8.3 다중 회귀 분석 (관중 수 예측 모델)

In [ ]:
# 다중 회귀 분석: 날씨 요인이 관중에 미치는 영향
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

# K리그 데이터 준비 (날씨 정보가 있는 데이터만)
df_regression = df_k_weather.copy()

# 범주형 변수 인코딩
le_league = LabelEncoder()
le_weather = LabelEncoder()

df_regression['LEAGUE_ENCODED'] = le_league.fit_transform(df_regression['LEAGUE_NAME'])
df_regression['WEATHER_ENCODED'] = le_weather.fit_transform(df_regression['Weather'])

# 특성과 타겟 분리
X = df_regression[['LEAGUE_ENCODED', 'Temperature', 'Humidity', 'WEATHER_ENCODED']]
y = df_regression['Audience_Qty']

# 회귀 모델 학습
model = LinearRegression()
model.fit(X, y)

# 결과 출력
print("=== 다중 회귀 분석 결과 ===\n")
print(f"R² (결정계수): {model.score(X, y):.4f}")
print(f"절편 (Intercept): {model.intercept_:.2f}")
print("\n회귀 계수 (Coefficients):")
print(f"  리그 (K리그1=0, K리그2=1): {model.coef_[0]:.2f}")
print(f"  온도 (°C): {model.coef_[1]:.2f}")
print(f"  습도 (%): {model.coef_[2]:.2f}")
print(f"  날씨: {model.coef_[3]:.2f}")

print("\n해석:")
print(f"- 온도가 1°C 증가하면 관중이 약 {model.coef_[1]:.0f}명 {'증가' if model.coef_[1] > 0 else '감소'}")
print(f"- 습도가 1% 증가하면 관중이 약 {abs(model.coef_[2]):.0f}명 {'증가' if model.coef_[2] > 0 else '감소'}")
print(f"- R² = {model.score(X, y):.4f} → 모델이 관중 변동의 약 {model.score(X, y)*100:.1f}%를 설명")

# 9. 최종 인사이트 및 가설 검증 결과

## 📊 주요 발견 사항

### 1. 리그 규모와 관중 동원력
- **J리그 (3단계 승강제)**
  - J1, J2, J3 총 3,428경기 (2023-2025)
  - 다층 승강제 시스템으로 더 많은 팀과 경기 운영
  
- **K리그 (2단계 승강제)**
  - K리그1, K리그2 총 1,431경기
  - J리그 대비 제한적인 리그 구조

### 2. 순위와 관중의 관계
- **상관관계 분석 결과**:
  - K리그1/K리그2 모두에서 순위와 관중 수 간 상관관계 존재
  - Spearman 상관계수를 통해 순위가 높을수록 관중이 많은 경향 확인
  - 단, 팀별 팬덤 규모와 지역적 요인이 더 큰 영향

- **순위권별 관중 차이**:
  - 상위권 팀이 중/하위권 대비 높은 평균 관중 기록
  - 그러나 특정 인기 팀은 순위와 무관하게 높은 관중 동원

### 3. 날씨와 환경 요인
- **온도**: 적정 온도(15-25°C)에서 관중 수가 높은 경향
- **날씨 유형**: 맑은 날씨가 비/흐림 대비 관중 동원에 유리
- **요일 효과**: 주말(토, 일) 경기가 평일 대비 관중 수 2배 이상
- **다중 회귀 분석**: 날씨 요인만으로는 관중 변동의 일부만 설명 가능

### 4. 시즌 단계별 패턴
- **ANOVA 검정 결과**: 시즌 초반/중반/후반 간 관중 수 차이 존재
- **시즌 초반**: 기대감과 호기심으로 상대적으로 높은 관중
- **시즌 후반**: 승강 경쟁이 치열한 경우 관중 증가 가능

### 5. 경기장 요인
- **대형 경기장 (서울, 수원, 전주 등)**: 높은 평균 관중
- **지역 밀착형 소형 경기장**: 상대적으로 낮은 관중이나 안정적 팬베이스

## 🎯 가설 검증 결과

### Main Hypothesis: 리그 유동성과 관중의 상관관계
> "리그 구성 팀의 빈번한 변화(High Churn Rate)는 총 관중 수를 증가시킨다"

- **부분 검증**: J리그의 3단계 승강제는 K리그보다 많은 경기와 팀을 운영
- **한계**: 현재 데이터만으로는 승강제 "전후" 비교 불가
- **추가 필요**: 승격/강등 팀의 전년 대비 관중 변화 추적 데이터

### Sub-Hypothesis A: 흥미의 정량화
> "경기 내 물리적 활동량이 높은 경기는 관중 재방문율을 높인다"

- **데이터 부족**: Physical Metrics (스프린트, 활동 거리) 데이터 미확보
- **차후 분석 필요**: 활동량 데이터 수집 후 재검증

### Sub-Hypothesis B: 승격 팀의 언더독 효과
> "승격한 팀은 기대 심리로 높은 관중 증가율을 기록한다"

- **데이터 부족**: 연도별 승격/강등 팀 구분 및 전년 대비 비교 데이터 필요
- **차후 분석 방향**: 팀별 시계열 분석 및 승격 전후 YoY 비교

## 💡 핵심 인사이트

1. **팀 브랜드 파워 > 순위**: 순위보다 팀 자체의 인기도가 관중에 더 큰 영향
2. **날씨는 보조 요인**: 날씨는 관중에 영향을 주지만 결정적 요인은 아님
3. **주말 > 평일**: 요일이 관중 동원에 가장 강력한 영향
4. **K리그1 > K리그2**: 통계적으로 유의미한 티어 간 관중 차이
5. **시즌 초반 효과**: 시즌 개막 시 관중 동원력이 높음

## 🔬 향후 연구 방향

1. **승격/강등 팀 추적 분석**: 연도별 팀 순위 변화와 관중 YoY 비교
2. **Physical Metrics 통합**: 경기 활동량 데이터 수집 및 상관관계 분석
3. **EPL/EFL Case Study**: 영국 리그 승격 팀의 "Promotion Bump" 효과 벤치마킹
4. **예측 모델 고도화**: 머신러닝 기반 관중 수 예측 모델 개발
5. **K3 리그 시뮬레이션**: 승강제 확대 시나리오별 관중 증가 효과 추정

---

## 📌 머신러닝 예측 모델

관중 수 예측을 위한 머신러닝 모델링은 별도 노트북에서 진행됩니다.

👉 **[modeling.ipynb](modeling.ipynb)**로 이동하여 확인하세요.

**modeling.ipynb 주요 내용**:
- 5가지 ML 모델 학습 및 비교 (Linear Regression, Random Forest, XGBoost, LightGBM, Gradient Boosting)
- 특성 공학 (17개 예측 변수)
- 특성 중요도 분석
- 예측 결과 시각화
- 모델 성능 평가 및 실무 활용 방안

In [ ]:
!pip install add xgboost

'uv'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
